In [1]:
#load libraries 
import time,os
from matplotlib.offsetbox import AnchoredText
import sys,pandas as pd, matplotlib , matplotlib.pyplot as plt, matplotlib.lines , numpy as np,cupy as cp, math, pylab
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
#import cudf
#import ROOT
#from ROOT import TFile

#import root_pandas

#from toggle_cell import hide_cell 
import uproot3
%matplotlib inline

#presets for plot style
from scipy.constants import golden
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (7, 7/golden),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

viridis = cm.get_cmap('viridis', 12)
inferno = cm.get_cmap('inferno', 12)
autumn = cm.get_cmap('autumn', 12)

#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)
rc = plt.rc
rc('text', usetex=False)
import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=False)
rc('font', size=15)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)
import root_pandas,glob

import gc

def read_n(pattern, n):
    if not '*' in pattern:
        a = root_pandas.read_root(pattern,chunksize=int(n))
    else:
        a = root_pandas.read_root(glob.glob(pattern),chunksize=int(n))
    for b in a:
        del a
        print(len(b))
        return b

from scipy.optimize import curve_fit

image_dir='/home/sebouh/dihadron-analysis-note/images/'

import misc_util

/home/sebouh/.local/lib/python3.8/site-packages/uproot3/__init__.py:127: FutureWarning: Consider switching from 'uproot3' to 'uproot', since the new interface became the default in 2020.

    pip install -U uproot

In Python:

    >>> import uproot
    >>> with uproot.open(...) as file:
    ...

  warnings.warn(
/home/sebouh/.local/lib/python3.8/site-packages/awkward0/__init__.py:12: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  warnings.warn(


Welcome to JupyROOT 6.22/06


In [2]:
mc_dihadrons_outb = root_pandas.read_root(glob.glob("/home/sebouh/tuples_hadd/mc_dihadrons_outb.root"),'dihadrons')
mc_dihadrons_inb = root_pandas.read_root(glob.glob("/home/sebouh/tuples_hadd/mc_dihadrons_inb.root"),'dihadrons')

KeyboardInterrupt: 

In [13]:
pi_pi_mc_inb = mc_dihadrons_inb.query("h1_cm_pt>0.3 and h2_cm_pt>0.3"\
        + " and abs(h2_pid) == 211 and abs(h1_pid) ==211 and h2_truth_z < 0.5")
pi_pi_mc_outb = mc_dihadrons_outb.query("h1_cm_pt>0.3 and h2_cm_pt>0.3"\
        + " and abs(h2_pid) == 211 and abs(h1_pid) ==211 and h2_truth_z < 0.5")

pi_p_mXcut_mc_inb = mc_dihadrons_inb.query("h1_cm_pt>0.3 and h2_cm_pt>0.3"\
        + " and h2_pid == 2212 and mx_eh1x_truth > 1.665 and abs(h1_pid) ==211 and h2_truth_z < 0.5")
pi_p_mXcut_mc_outb = mc_dihadrons_outb.query("h1_cm_pt>0.3 and h2_cm_pt>0.3"\
        + " and h2_pid == 2212 and mx_eh1x_truth > 1.665 and abs(h1_pid) ==211 and h2_truth_z < 0.5")

In [22]:
bins = 50
def offset(a):
    return a+2*np.pi*(a<-np.pi/2)-2*np.pi*(a>=3*np.pi/2)
for i in (0,1):
    

    q = ("diff_rap_cm_truth > %s and diff_rap_cm_truth < %s" %(1.5,2.5)) 
    q += " and h2_truth_z<0.5 and h1_truth_p>0 and h2_truth_p>0  and h2_truth_z<h1_truth_z "
    q += " and h1_truth_cm_pt>0.3 and h2_truth_cm_pt>0.3 and abs(h1_truth_pid) ==211 and h1_truth_z > 0.4"

    for p in 'h1 h2 e'.split():
        q += " and part_truth_th < 45*3.14159/180 and part_truth_th > 5*3.14159/180".replace("part",p)

    
    if i == 1:
        q+= " and abs(h2_truth_pid) == 211"
        dfs = [df.query(q) for df in [mc_dihadrons_inb,mc_dihadrons_outb]]
    else :
        q+= " and h2_truth_pid == 2212"
        dfs = [df.query(q) for df in [mc_dihadrons_inb,mc_dihadrons_outb]]
    ys = []
    dys = []
    for df in dfs:
        df.eval('mx_eh1x_truth = sqrt((E+e_truth_p+.9383)**2-(-e_truth_px-h1_truth_px)**2'
                      +'-(-e_truth_py-h1_truth_py)**2-(E-e_truth_pz-h1_truth_pz)**2)')
        ysmc, x = np.histogram(offset(df.query("h2_cm_ph!=0").diff_phi_cm_truth), bins=bins, range=(-np.pi/2,3*np.pi/2))
        ysmc0 = ysmc
        
        x = np.add(x[1:],x[:-1])/2
        denom = sum(ysmc)*(x[1]-x[0])
        dysmc = np.sqrt(ysmc)
        ysmc=np.divide(ysmc,denom)
        dysmc=np.divide(dysmc,denom)
    
        
        yt,x = np.histogram(offset(df.diff_phi_cm_truth), bins=bins, range=(-np.pi/2,3*np.pi/2))
        x = np.add(x[1:],x[:-1])/2

        yt0 = yt
        denom = sum(yt)*(x[1]-x[0])
        dyt=np.sqrt(yt)/denom
        yt = yt/denom
        
        ys.append(yt/ysmc)
        dys.append(np.sqrt(ysmc0*(yt0-ysmc0))/yt0**(3/2))
    print("x =",list(x))    
    print("y = ",list((ys[0]/dys[0]**2+ys[1]/dys[1]**2)/(1/dys[0]**2+1/dys[1]**2)/(2*np.pi)))
    print("dy = ", list(np.sqrt(1/(1/dys[0]**2+1/dys[1]**2))/(2*np.pi)))

x = [-1.5079644737231006, -1.3823007675795091, -1.2566370614359172, -1.1309733552923253, -1.0053096491487339, -0.879645943005142, -0.7539822368615503, -0.6283185307179585, -0.5026548245743668, -0.37699111843077515, -0.25132741228718336, -0.12566370614359157, 1.1102230246251565e-16, 0.1256637061435919, 0.2513274122871837, 0.3769911184307754, 0.5026548245743672, 0.6283185307179588, 0.7539822368615505, 0.8796459430051422, 1.0053096491487339, 1.1309733552923258, 1.2566370614359175, 1.3823007675795091, 1.507964473723101, 1.6336281798666927, 1.7592918860102844, 1.8849555921538763, 2.0106192982974678, 2.1362830044410597, 2.2619467105846516, 2.387610416728243, 2.513274122871835, 2.638937829015427, 2.7646015351590183, 2.8902652413026098, 3.0159289474462017, 3.1415926535897936, 3.2672563597333855, 3.392920065876977, 3.5185837720205684, 3.6442474781641603, 3.769911184307752, 3.895574890451344, 4.0212385965949355, 4.1469023027385274, 4.272566008882119, 4.39822971502571, 4.523893421169303, 4.649557